rouge 
bertscore

f1radgraph

bartscore
alignscore

In [110]:
import os
import json
import numpy as np
import pandas as pd

from rouge import Rouge

In [19]:
generated_dir = ''
reference_dir = '../../autoclin24-data/public/test'
score_dir = ''

# reference_dir = os.path.join('/app/input/', 'ref')
# prediction_dir = os.path.join('/app/input/', 'res')
# score_dir = '/app/output/'

In [78]:
print('Reading generated texts...')
generated = pd.read_csv(os.path.join(generated_dir, 'submission.csv'))
reference = pd.read_csv(os.path.join(reference_dir, 'discharge_target.csv.gz'))

# covert all elements to string
generated['discharge_instructions'] = generated['discharge_instructions'].astype(str)
reference['discharge_instructions'] = reference['discharge_instructions'].astype(str)

generated['brief_hospital_course'] = generated['brief_hospital_course'].astype(str)
reference['brief_hospital_course'] = reference['brief_hospital_course'].astype(str)

# convert to multiline strings
generated['discharge_instructions'] = generated['discharge_instructions'].str.replace('\n', ' ')
reference['discharge_instructions'] = reference['discharge_instructions'].str.replace('\n', ' ')

generated['brief_hospital_course'] = generated['brief_hospital_course'].str.replace('\n', ' ')
reference['brief_hospital_course'] = reference['brief_hospital_course'].str.replace('\n', ' ')

print('Done.')

Reading generated texts...
Done.


In [131]:
def calculate_scores(generated, reference):
    scores = {}

    rouge = Rouge()
    discharge_instructions_scores = rouge.get_scores(list(generated['discharge_instructions']), list(reference['discharge_instructions']), avg=True)
    brief_hospital_course_scores = rouge.get_scores(list(generated['brief_hospital_course']), list(reference['brief_hospital_course']), avg=True)

    scores['rouge1'] = (discharge_instructions_scores['rouge-1']['f'] + brief_hospital_course_scores['rouge-1']['f'])/2
    scores['rouge2'] = (discharge_instructions_scores['rouge-2']['f'] + brief_hospital_course_scores['rouge-2']['f'])/2
    scores['rougeL'] = (discharge_instructions_scores['rouge-l']['f'] + brief_hospital_course_scores['rouge-l']['f'])/2

    return scores
    

In [130]:
scores = calculate_scores(generated, reference)

with open(os.path.join(score_dir, 'scores.json'), 'w') as score_file:
    score_file.write(json.dumps(scores))